In [2]:
import numpy as np
from sympy import symbols, diff, lambdify

def get_user_input():
    """Get function and initial point from user"""
    print("Enter your quadratic function in terms of x and y (e.g., x - y + 2*x**2 + 2*x*y + y**2):")
    func_str = input("f(x, y) = ")
    
    print("\nEnter initial point (space-separated, e.g., 0 0):")
    while True:
        x0_input = input("x0 = ").split()
        if len(x0_input) == 2:
            try:
                x0 = [float(num) for num in x0_input]
                return func_str, np.array(x0)
            except ValueError:
                print("Please enter valid numbers!")
        else:
            print("Please enter exactly 2 numbers!")

def prepare_function(func_str):
    """Convert string to computable function and compute gradient"""
    x, y = symbols('x y')
    f_expr = eval(func_str)
    
    # Compute gradient symbolically
    df_dx = diff(f_expr, x)
    df_dy = diff(f_expr, y)
    
    # Convert to numerical functions
    f = lambdify((x, y), f_expr, 'numpy')
    grad_f = lambdify((x, y), [df_dx, df_dy], 'numpy')
    
    return f, grad_f

def conjugate_gradient(f, grad_f, x0, max_iter=1000, tol=1e-6):
    """Conjugate Gradient optimization implementation"""
    x = x0.copy()
    grad = np.array(grad_f(*x))
    d = -grad  # Initial search direction
    trajectory = [x.copy()]
    
    for k in range(max_iter):
        # Line search (for quadratic functions, we can compute optimal alpha)
        grad_current = np.array(grad_f(*x))
        grad_next = np.array(grad_f(*(x + d)))
        
        # For exact line search in quadratic case:
        alpha = - (grad_current @ d) / (d @ (grad_next - grad_current)) if np.any(grad_next - grad_current) else 0.1
        
        # Update solution
        x_new = x + alpha * d
        grad_new = np.array(grad_f(*x_new))
        
        # Store trajectory
        trajectory.append(x_new.copy())
        
        # Check convergence
        if np.linalg.norm(grad_new) < tol:
            print(f"\nConverged in {k+1} iterations!")
            return x_new, trajectory
        
        # Fletcher-Reeves beta coefficient
        beta = (grad_new @ grad_new) / (grad_current @ grad_current)
        
        # Update search direction
        d = -grad_new + beta * d
        
        # Prepare for next iteration
        x, grad = x_new, grad_new
    
    print("\nMaximum iterations reached without convergence!")
    return x, trajectory

def main():
    print("Conjugate Gradient Optimization")
    print("------------------------------")
    
    # Get user input
    func_str, x0 = get_user_input()
    
    # Prepare the function and its gradient
    f, grad_f = prepare_function(func_str)
    
    # Run optimization
    print("\nRunning optimization...")
    solution, trajectory = conjugate_gradient(f, grad_f, x0)
    
    # Display results
    print("\n=== Optimization Results ===")
    print(f"Initial point: {x0}")
    print(f"Optimal solution: {solution}")
    print(f"Function value at solution: {f(*solution):.6f}")
    
    # Print optimization path
    print("\nOptimization path:")
    for i, point in enumerate(trajectory):
        print(f"Iter {i}: x = {point}, f(x) = {f(*point):.6f}")

if __name__ == "__main__":
    main()

Conjugate Gradient Optimization
------------------------------
Enter your quadratic function in terms of x and y (e.g., x - y + 2*x**2 + 2*x*y + y**2):


f(x, y) =   x - y + 2*x**2 + 2*x*y + y**2



Enter initial point (space-separated, e.g., 0 0):


x0 =  0 0



Running optimization...

Converged in 2 iterations!

=== Optimization Results ===
Initial point: [0. 0.]
Optimal solution: [-1.   1.5]
Function value at solution: -1.250000

Optimization path:
Iter 0: x = [0. 0.], f(x) = 0.000000
Iter 1: x = [-1.  1.], f(x) = -1.000000
Iter 2: x = [-1.   1.5], f(x) = -1.250000
